## Process Tetrahymena's UTRs

In [31]:
import os
import pandas as pd
import math
utrSize = 20
numberOfPositions = utrSize // 3

filein = open('Tetrahymena_thermophila.utr20','r')
stops = ['TGA']
#Calculate frequency of stops in the 3' UTRs

tandems = []

filein.readline()
for line in filein:
    num, name, realStop, utr, seq = line.split()
    utr = utr.upper()
    tandem = 0
    for i in range(numberOfPositions):
        current = utr[i*3:i*3+3]
        if current in stops:
            tandem = i + 1
            break
    found = 0
    if tandem:
        found = 1
    tandems.append([name, realStop,tandem,found,1])

filein.close()

tandemDF = pd.DataFrame(tandems, columns=['gene','stop','tandemPos',
                                         'hasTandem','counter'])

tandemDF.set_index('gene',inplace=True)
tandemDF.head()

,stop,tandemPos,hasTandem,counter
gene,,,,
TTHERM_01431530,TGA,0,0,1
TTHERM_001431536,TGA,0,0,1
TTHERM_01431550,TGA,0,0,1
TTHERM_01431560,TGA,0,0,1
TTHERM_01431570,TGA,0,0,1


In [32]:
CAIs = pd.read_csv('tetCodonBias.tsv',sep='\t',index_col =0)
CAIs.set_index('gene',inplace=True)
CAIs.head()

,CAI
gene,
TTHERM_000994399,0.401820
TTHERM_00994140,0.405599
TTHERM_00994360,0.400522
TTHERM_00994330,0.359434
TTHERM_00994200,0.394859


In [40]:
allData = tandemDF.join(CAIs, how='left')[['hasTandem','counter','CAI']]
allData.head()

,hasTandem,counter,CAI
gene,,,
TTHERM_01431530,0,1,0.519211
TTHERM_001431536,0,1,0.341275
TTHERM_01431550,0,1,0.308122
TTHERM_01431560,0,1,0.335164
TTHERM_01431570,0,1,0.451342


In [41]:
allData.describe()

,hasTandem,counter,CAI
count,26498.000000,26498.0,26498.000000
mean,0.114952,1.0,0.419189
std,0.318970,0.0,0.078755
min,0.000000,1.0,0.110821
25%,0.000000,1.0,0.379082
50%,0.000000,1.0,0.402070
75%,0.000000,1.0,0.435587
max,1.000000,1.0,0.946254


In [42]:
# get the quartiles of the CAI distribution
bins = np.array([0,
                 allData.describe()['CAI']['25%'],
                 allData.describe()['CAI']['50%'],
                 allData.describe()['CAI']['75%'],
                 1])

In [44]:
import numpy as np

#use the quartiles 
#bins =  np.array([0,0.379082,0.402070,0.435587,1])
ind = np.digitize(allData['CAI'],bins)

allData['ind'] = ind

result = allData.groupby(by='ind').sum()[['hasTandem','counter']]
result

,hasTandem,counter
ind,,
1,650,6625
2,703,6624
3,722,6624
4,971,6625


In [46]:
obsTSC = result['hasTandem'][4]
obsNo = result['counter'][4] - result['hasTandem'][4]
expFreq = (allData.sum()['hasTandem']/allData.sum()['counter'])
expTSC = result['counter'][4] * expFreq
expNo = result['counter'][4] * (1-expFreq)

print(obsTSC, obsNo)
print(expTSC, expNo)

971 5654
761.5574760359273 5863.442523964072


In [49]:
from scipy.stats import chisquare
obs = [obsTSC,obsNo]
exp = [expTSC,expNo]
chisquare(obs,exp)

Power_divergenceResult(statistic=65.08190402881675, pvalue=7.184884221225469e-16)